# Forex testing with the backtesting library and other experiments

In [1]:
#from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
from datetime import datetime

## Import forex from spreadsheet and convert timestamp strings to an index

In [48]:
forexData = pd.read_csv("forexData.csv")
#forexJson = forexData.json()
price = pd.DataFrame(forexData)
#datetime.strptime(dt_string, "%Y.%m.%d %H:%M")
price["Date"] = price["Date"].apply(lambda x: datetime.strptime(x, "%Y.%m.%d %H:%M"))
price['Date']=price["Date"].apply(lambda x: pd.Timestamp(x, unit='s'))
price['Date']=pd.DatetimeIndex(price["Date"])

price = price.set_index('Date')
price

,Open,High,Low,Close,Volume,NotSure
Date,,,,,,
2020-01-27 01:05:00,1.10274,1.10313,1.10272,1.10302,264,0
2020-01-27 01:10:00,1.10302,1.10308,1.10275,1.10296,219,0
2020-01-27 01:15:00,1.10296,1.10329,1.10296,1.10322,401,0
2020-01-27 01:20:00,1.10322,1.10327,1.10310,1.10317,322,0
2020-01-27 01:25:00,1.10318,1.10340,1.10318,1.10338,359,0
...,...,...,...,...,...,...
2021-05-28 23:35:00,1.21915,1.21926,1.21915,1.21917,73,0
2021-05-28 23:40:00,1.21916,1.21939,1.21905,1.21936,140,0
2021-05-28 23:45:00,1.21936,1.21940,1.21917,1.21921,135,0


## testing with backtesting.py

In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from statistics import mean

def mySMA(myArr, period):
    return pd.Series(myArr).rolling(period).mean()
            
def myEMA(myArr, period):
    return pd.Series(myArr).ewm(adjust=False, span=period).mean()
    
def myMacd(myArr):
    twelve = myEMA(myArr, 12) 
    twentySix = myEMA(myArr, 26)
    macd = twelve-twentySix
#     print(macd, twelve, twentySix)
    return macd

def myMacdSignal(myArr):
    return myEMA(myMacd(myArr), 9)

def myMacdHist(myArr):
    return myMacd(myArr) - myMacdSignal(myArr)

class SmaCross(Strategy):
    
    def init(self):
        profitLossAmnt = 0
        closePrice = self.data.Close
        openPrice = self.data.Open
        #print("buy and hold range", tempprice[0], tempprice[-1])
        self.emaOpen4 = self.I(myEMA, openPrice, 4)
        self.emaOpen12 = self.I(myEMA, openPrice, 12)
        self.emaClose4 = self.I(myEMA, closePrice, 4)
        self.emaClose12 = self.I(myEMA, closePrice, 12)
#         self.ma3 = self.I(myEMA, tempprice, 100)

        #self.macd = self.I(myMacd, tempprice)
        #self.macdSignal = self.I(myMacdSignal, tempprice)
        #self.macdHist = self.I(myMacdHist, tempprice)

        
    def next(self):
        if crossover(self.emaOpen4, self.emaClose4):
            self.buy()
        elif crossover(self.emaClose4, self.emaOpen4):
            self.sell()

bt = Backtest(price, SmaCross,
              exclusive_orders=True)
stats = bt.run()
print(stats._trades["PnL"].sum())
print(stats._trades)
stats._trades.to_csv("trades.csv")

bt.plot()

# grouping data by when a run starts

In [55]:
maLength = 8
maOpenValues = myEMA(price["Open"], maLength)
maCloseValues = myEMA(price["Close"], maLength)
df = pd.DataFrame(columns=["hour", "runLength"])
counter = 1
for i in range(1,len(price)):
    closeOverBefore = 1 if (maOpenValues[i-1] <= maCloseValues[i-1]) else 0
    closeOver = 1 if (maOpenValues[i] <= maCloseValues[i]) else 0
#     print(i, price.index[i], price.index[i].hour, price["Close"][i], maOpenValues[i], maCloseValues[i], closeOver)
    if(closeOverBefore != closeOver):
        if(counter > 10):
            df = df.append({"hour":price.index[i].hour + 1, "runLength":counter}, ignore_index=True)
        counter = 1
    else:
        counter = counter + 1
groupedDf = df.groupby("hour").count()
print(groupedDf)
df.to_csv("run.csv")


      runLength
hour           
1            89
2            80
3           102
4           108
5           117
6           107
7           103
8           109
9            86
10           88
11           83
12          122
13           97
14           89
15           95
16          104
17           91
18          110
19           90
20          101
21          102
22          102
23           96
24          105
